In [4]:
!pip install -U "langchain==0.3.*" "langchain-core==0.3.*" "langchain-community==0.3.*" "langgraph==0.3.*" "langchain-huggingface" "huggingface_hub" "sentence-transformers" wikipedia -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.4.1 requires langchain-community<1.0.0,>=0.4.0, but you have langchain-community 0.3.31 which is incompatible.
langchain-experimental 0.4.1 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.83 which is incompatible.
langchain-classic 1.0.1 requires langchain-core<2.0.0,>=1.2.5, but you have langchain-core 0.3.83 which is incompatible.
langchain-classic 1.0.1 requires langchain-text-splitters<2.0.0,>=1.1.0, but you have langchain-text-splitters 0.3.11 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/LangGraph/'

# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

# prompt llm output parser

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
prompt_value = prompt.invoke({"topic": "ice cream"})
"""
messages=[HumanMessage(content='tell me a short joke about ice cream')]
"""
print(prompt_value.to_string())
"""
Human: tell me a short joke about ice cream
"""

model = ChatOpenAI()
message = model.invoke(prompt_value)
"""
content='Why did the ice cream go to therapy? \n\nBecause it had too many scoops of emotions!'
"""

# from langchain.llms import OpenAI
# llm = OpenAI(model="gpt-3.5-turbo-instruct")
# llm_message = llm.invoke(prompt_value)
"""
Why did the ice cream go to therapy?

Because it was having a meltdown!
"""

output_parser = StrOutputParser()
print(output_parser.invoke(message))
"""
Why did the ice cream go to therapy?

Because it had a meltdown!
"""

# similar to unix pipe operator
chain = prompt | model | output_parser

result = chain.invoke({"topic": "ice cream"})
print(result)
"""
Why did the ice cream go to therapy?

Because it had too many sprinkles of anxiety!
# """

In [4]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

# 1. 프롬프트 템플릿 정의
# {topic} 자리에 사용자가 원하는 주제를 넣을 수 있는 대화형 프롬프트입니다.
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

# 프롬프트 단독 실행 테스트 (입력값을 넣어 메시지 객체로 변환)
prompt_value = prompt.invoke({"topic": "ice cream"})
"""
messages=[HumanMessage(content='tell me a short joke about ice cream')]
"""
print(prompt_value.to_string())
"""
Human: tell me a short joke about ice cream
"""

# 2. 모델 설정 (HuggingFace 엔드포인트 사용)
# gpt-oss-20b 모델을 호출하는 엔드포인트를 생성합니다.
llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="text-generation")
model = ChatHuggingFace(llm=llm_ep)

# 모델 단독 실행 테스트 (프롬프트 결과값을 모델에 전달)
# 결과는 AIMessage 객체 형태로 반환됩니다. (내용은 content 속성에 담김)
message = model.invoke(prompt_value)
"""
content='Why did the ice cream go to therapy? \n\nBecause it had too many scoops of emotions!'
"""

# from langchain.llms import OpenAI
# llm = OpenAI(model="gpt-3.5-turbo-instruct")
# llm_message = llm.invoke(prompt_value)
"""
Why did the ice cream go to therapy?

Because it was having a meltdown!
"""

# 3. 출력 파서 설정
# 모델의 응답(객체)에서 텍스트(string)만 깔끔하게 추출해주는 파서입니다.
output_parser = StrOutputParser()
# 파서 단독 실행 테스트
print(output_parser.invoke(message))
"""
Why did the ice cream go to therapy?

Because it had a meltdown!
"""

# --- 4. LCEL(LangChain Expression Language) 체인 생성 ---
# | 연산자를 사용하여 각 단계를 하나로 묶습니다. (Unix 파이프와 동일한 원리)
# 데이터 흐름: 입력(dict) -> 프롬프트 -> 모델 -> 출력 파서 -> 최종 문자열
# similar to unix pipe operator
chain = prompt | model | output_parser

# 5. 체인 실행 (전체 과정을 한 번에 수행)
result = chain.invoke({"topic": "ice cream"})
print(result)
"""
Why did the ice cream go to therapy?

Because it had too many sprinkles of anxiety!
# """

Human: tell me a short joke about ice cream
Why did the ice cream start a band? Because it wanted to be a “scoop” of music!
Why did the ice cream keep a diary?  
Because it was always “melting” under pressure!


'\nWhy did the ice cream go to therapy?\n\nBecause it had too many sprinkles of anxiety!\n# '

# prompt llm function call

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import (
    JsonOutputFunctionsParser,
    JsonKeyOutputFunctionsParser)
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough


prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()

# function call
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)
# print(chain.invoke({"topic": "ice cream"}, config={}))
"""
content='' additional_kwargs={'function_call': {'arguments': '{\n  "setup": "Why did the ice cream go to therapy?",\n  "punchline": "Because it was feeling a little melty!"\n}', 'name': 'joke'}}
"""

# chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions) | JsonOutputFunctionsParser()
# print(chain.invoke({"topic": "ice cream"}))
"""
{'setup': 'Why did the ice cream go to therapy?', 'punchline': 'Because it had too many sprinkles of anxiety!'}
"""

# chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions) | JsonKeyOutputFunctionsParser(key_name="setup")
# print(chain.invoke({"topic": "ice cream"}))
"""
Why did the ice cream go to therapy?
"""

map_ = RunnableParallel(topic=RunnablePassthrough())
chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)
print(chain.invoke("ice cream"))
"""
Why did the ice cream go to therapy?
"""

chain = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)
print(chain.invoke("ice cream"))
"""
Why did the ice cream break up with the cone?
"""

In [12]:
pip install langchain-core

In [16]:
import os
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. HF_TOKEN 설정 (선택 사항이지만 권장)
# Colab 왼쪽 열쇠 아이콘(Secrets)에서 HF_TOKEN을 추가하거나 직접 입력하세요.
# os.environ["HF_TOKEN"] = "your_huggingface_token_here"

# 2. 모델 설정 (대화형 태스크에 최적화된 Llama-3.2 사용)
repo_id = "meta-llama/Llama-3.2-3B-Instruct"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    task="text-generation", # 내부 엔진용
    max_new_tokens=512,
    temperature=0.1
)

# Chat 모델로 래핑 (conversational 태스크 대응)
model = ChatHuggingFace(llm=llm)

# 3. JSON 형식을 강력하게 요구하는 프롬프트
prompt = ChatPromptTemplate.from_template(
    """You are a joke generator.
    Respond ONLY with a JSON object containing 'setup' and 'punchline' keys.
    Do not include any other text or explanation.

    Topic: {topic}"""
)

# 4. 체인 구성
chain = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model
    | JsonOutputParser()
)

# 5. 실행
try:
    print("--- 결과 확인 ---")
    result = chain.invoke("ice cream")
    print(f"성공! 결과: {result}")
    print(f"Setup: {result.get('setup')}")

except Exception as e:
    print(f"에러 발생: {e}")
    # 실패 시 모델이 보낸 생문자열 확인 (디버깅용)
    debug_chain = {"topic": RunnablePassthrough()} | prompt | model
    debug_res = debug_chain.invoke("ice cream")
    print(f"모델 원본 응답: {debug_res.content}")

--- 결과 확인 ---
성공! 결과: {'setup': 'Why did the ice cream go to therapy?', 'punchline': 'Because it had a meltdown!'}
Setup: Why did the ice cream go to therapy?


# Retrieval-augmented generation(RAG)

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_community.vectorstores.faiss import FAISS

# It requires `pip install langchain openai faiss-cpu tiktoken`
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=embedding_model
)
# save
vectorstore.save_local("faiss_index")

vectorstore_new = FAISS.load_local("faiss_index", embedding_model)

retriever = vectorstore_new.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke("where did harrison work?"))
"""
Harrison worked at Kensho.
"""

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke({"question": "where did harrison work", "language": "italian"}))
"""
Harrison ha lavorato a Kensho.
"""

# # itemgetter example
# from operator import itemgetter
# # Suppose we have a dictionary
# person = {'name': 'Alice', 'age': 30, 'job': 'Engineer'}
# # We can use itemgetter to create a function that fetches the 'name' from a dictionary
# get_name = itemgetter('name')
# # Now, when we use this function with our dictionary
# name = get_name(person)
# print(name)  # Output: Alice

In [20]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 68.1 MB/s eta 0:00:00


In [21]:
import os
from operator import itemgetter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint, ChatHuggingFace
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# 1. 임베딩 모델 설정 (로컬 HuggingFace 모델)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 2. 벡터스토어 생성 및 로드 (FAISS)
# 주의: allow_dangerous_deserialization=True는 로컬에서 직접 만든 인덱스를 로드할 때 필요합니다.
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=embeddings
)
vectorstore.save_local("faiss_index")

vectorstore_new = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)
retriever = vectorstore_new.as_retriever()

# 3. LLM 설정 (Hugging Face 오픈소스 모델)
# llm_ep = HuggingFaceEndpoint(
#     repo_id="meta-llama/Llama-3.2-3B-Instruct", # 또는 "Qwen/Qwen2.5-7B-Instruct"
#     task="text-generation",
#     max_new_tokens=512,
#     temperature=0.1
# )
llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="text-generation")
model = ChatHuggingFace(llm=llm_ep)

# --- 시나리오 1: 단순 질문 전달 ---
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print("--- Scenario 1 Result ---")
print(chain.invoke("where did harrison work?"))


# --- 시나리오 2: itemgetter를 이용한 다중 파라미터 전달 ---
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

# [핵심] 입력 데이터 딕셔너리에서 원하는 키값만 뽑아서 각 컴포넌트에 전달합니다.
chain = (
    {
        "context": itemgetter("question") | retriever, # 질문만 뽑아서 리트리버에 전달
        "question": itemgetter("question"),           # 질문만 뽑아서 프롬프트에 전달
        "language": itemgetter("language"),           # 언어 설정만 뽑아서 프롬프트에 전달
    }
    | prompt
    | model
    | StrOutputParser()
)

print("\n--- Scenario 2 Result (Italian) ---")
print(chain.invoke({"question": "where did harrison work", "language": "italian"}))

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


--- Scenario 1 Result ---
Harrison worked at **Kensho**.

--- Scenario 2 Result (Italian) ---
Harrison ha lavorato a Kensho.


# Runnable Protocol

## Runnable Lambda

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOpenAI()
chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)
print(chain.invoke({"foo": "bar", "bar": "gah"}))
"""
content='3 + 9 = 12'
"""

In [22]:
import os
from operator import itemgetter
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# 1. 사용자 정의 함수 정의
# 각 함수는 입력값을 받아 특정 연산을 수행합니다.
def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

# 2. 모델 설정 (Hugging Face 오픈소스 모델)
# Llama-3 또는 Qwen2.5 같은 고성능 모델을 사용하면 계산 결과를 더 잘 설명합니다.
llm_ep = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    max_new_tokens=100,
    temperature=0.1
)
model = ChatHuggingFace(llm=llm_ep)

# 3. 프롬프트 정의
prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

# 4. 체인 구성 (LCEL)
# RunnableLambda를 사용해 일반 파이썬 함수를 체인의 구성 요소로 변환합니다.
chain = (
    {
        # "foo"의 길이를 계산하여 'a'에 할당
        "a": itemgetter("foo") | RunnableLambda(length_function),

        # "foo"와 "bar"의 길이를 곱하여 'b'에 할당
        "b": (
            {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
            | RunnableLambda(multiple_length_function)
        ),
    }
    | prompt
    | model
    | StrOutputParser() # 깔끔한 텍스트 출력을 위해 추가
)

# 5. 실행
# foo(3글자), bar(3글자) -> a=3, b=3*3=9 -> "what is 3 + 9"
print("--- 연산 결과 ---")
print(chain.invoke({"foo": "bar", "bar": "gah"}))

--- 연산 결과 ---
3 + 9 = 12


## Runnable Branch

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `Strawberry`, `Banana`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI()
    | StrOutputParser()
)

print(chain.invoke({"question": "What is the fruit that has red color?"}))
"""
Strawberry
"""

strawberry_chain = (
    PromptTemplate.from_template(
        """You are an expert about strawberry. \
Always answer questions starting with "As a Strawberry expert ... ". \
Respond to the following question:

# Question: {question}
# Answer:"""
    )
    | ChatOpenAI()
)

banana_chain = (
    PromptTemplate.from_template(
        """You are an expert about banana. \
Always answer questions starting with "As a Banana expert ... ". \
Respond to the following question:

# Question: {question}
# Answer:"""
    )
    | ChatOpenAI()
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)


from langchain.schema.runnable import RunnableBranch

# the first element is a condition (a lambda function) and
# the second element is the chain to execute if the condition is true
branch = RunnableBranch(
    (lambda x: "strawberry" in x["topic"].lower(), strawberry_chain), # type: ignore
    (lambda x: "banana" in x["topic"].lower(), banana_chain), # type: ignore
    general_chain,
)

# chain is invoked to classify the question, and its output is stored under the key topic.
# The original question is passed through unchanged under the key question.
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

print(full_chain.invoke({"question": "What is the fruit that has red color?"}))
print(full_chain.invoke({"question": "What is the fruit that has yellow color?"}))
print(full_chain.invoke({"question": "What is the fruit that has green color?"}))
"""
content='As a Strawberry expert, I can tell you that strawberries are the fruit that has a vibrant red color.'
content='As a Banana expert, the fruit that has a yellow color is the banana.'
content='The fruit that has a green color is typically an apple or a lime.'
"""

In [23]:
import os
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnableBranch

# 1. LLM 설정 (Hugging Face 오픈소스 모델)
# 분류 및 전문 답변 능력이 좋은 모델을 사용합니다.
llm_ep = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    temperature=0.1
)
model = ChatHuggingFace(llm=llm_ep)

# 2. 질문 분류 체인 (Classifier)
# 입력된 질문을 Strawberry, Banana, Other 중 하나로 분류합니다.
classification_chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `Strawberry`, `Banana`, or `Other`.
Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | model
    | StrOutputParser()
)

# 3. 각 주제별 전문 체인 (Sub-chains)
strawberry_chain = (
    PromptTemplate.from_template(
        """You are an expert about strawberry. Always answer questions starting with "As a Strawberry expert ... ".
Question: {question}
Answer:"""
    )
    | model
)

banana_chain = (
    PromptTemplate.from_template(
        """You are an expert about banana. Always answer questions starting with "As a Banana expert ... ".
Question: {question}
Answer:"""
    )
    | model
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:
Question: {question}
Answer:"""
    )
    | model
)

# 4. RunnableBranch 설정 (분기 로직)
# "topic" 결과값에 따라 어느 체인으로 갈지 결정합니다.
branch = RunnableBranch(
    (lambda x: "strawberry" in x["topic"].lower(), strawberry_chain),
    (lambda x: "banana" in x["topic"].lower(), banana_chain),
    general_chain,
)

# 5. 전체 체인 구성 (Full Chain)
# 먼저 분류(topic)를 수행하고, 원본 질문(question)과 함께 branch로 넘깁니다.
full_chain = {"topic": classification_chain, "question": lambda x: x["question"]} | branch

# 6. 실행 및 테스트
print("--- 결과 확인 ---")
res1 = full_chain.invoke({"question": "What is the fruit that has red color?"})
print(f"Red: {res1.content}")

res2 = full_chain.invoke({"question": "What is the fruit that has yellow color?"})
print(f"Yellow: {res2.content}")

res3 = full_chain.invoke({"question": "What is the fruit that has green color?"})
print(f"Green: {res3.content}")

--- 결과 확인 ---
Red: As a Strawberry expert, I can tell you that strawberries are indeed one of the fruits that have a red color. However, it's worth noting that strawberries are actually aggregate fruits, meaning that they're formed from multiple ovaries of a single flower. Each "seed" on the surface of a strawberry is actually an individual fruit, and they're all fused together to form the single fruit. But if you're thinking of a single fruit with a red color, strawberries are definitely one of the top contenders!
Yellow: I can't provide information on a specific fruit that has a yellow color, as the question doesn't specify a type of fruit. If you'd like to ask about strawberries, I can answer.

As a Strawberry expert, I can tell you that strawberries are typically red in color, but they can vary in color depending on the variety and ripeness. Some strawberries may have a yellow or white tint to their skin, but this is not a characteristic of the fruit itself.
Green: Unfortunately, t

## Runnable Parallel

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from operator import itemgetter
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough()
)

print(runnable.invoke({"num": 1}))
"""
{'passed': {'num': 1}}
"""

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

print(runnable.invoke({"num": 1}))
"""
{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}
"""

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel

model = ChatOpenAI()

joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

# easy to execute multiple Runnables in parallel
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

print(map_chain.invoke({"topic": "bear"}))
"""
{
    'joke': AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!"),
    'poem': AIMessage(content='In the dark woods, a bear roams free,\nA majestic creature, wild and full of mystery.')
}
"""

In [24]:
import os
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# --- 1. 데이터 가공 기본 (RunnablePassthrough & assign) ---
# 이 부분은 모델과 상관없이 LangChain의 핵심 로직이므로 동일하게 작동합니다.

# 입력받은 값을 그대로 'passed' 키에 담습니다.
runnable_basic = RunnableParallel(
    passed=RunnablePassthrough()
)
print("--- Basic Invoke ---")
print(runnable_basic.invoke({"num": 1})) # {'passed': {'num': 1}}

# assign을 사용해 기존 데이터를 유지하면서 새로운 값을 추가합니다.
runnable_complex = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)
print("\n--- Complex Invoke ---")
print(runnable_complex.invoke({"num": 1}))
# {'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}


# --- 2. 병렬 체인 실행 (Hugging Face 모델 연동) ---

# 모델 설정 (Llama-3.2 사용)
llm_ep = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    max_new_tokens=100,
    temperature=0.7
)
model = ChatHuggingFace(llm=llm_ep)

# 농담 체인과 시(Poem) 체인을 각각 정의합니다.
joke_chain = (
    ChatPromptTemplate.from_template("tell me a short joke about {topic}")
    | model
    | StrOutputParser() # 텍스트만 깔끔하게 추출
)
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}")
    | model
    | StrOutputParser()
)

# [핵심] RunnableParallel을 사용하여 두 체인을 묶습니다.
# invoke 하나로 joke와 poem이 동시에 생성됩니다.
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

print("\n--- Map Chain Invoke (Parallel) ---")
result = map_chain.invoke({"topic": "bear"})
print(f"Joke: {result['joke']}")
print(f"Poem: {result['poem']}")

--- Basic Invoke ---
{'passed': {'num': 1}}

--- Complex Invoke ---
{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

--- Map Chain Invoke (Parallel) ---
Joke: Why did the bear go to the doctor?

Because it had a grizzly cough!
Poem: Here's a 2-line poem about a bear:

A gentle bear in forest deep,
Roams freely, in a peaceful sleep.


# Multiple Chains

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

# Example1
prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

print(chain2.invoke({"person": "obama", "language": "english"}))
"""
Barack Obama, the 44th President of the United States, was born in Honolulu, Hawaii, which is located in the United States of America.
"""

# Example2
from langchain.schema.runnable import RunnablePassthrough

# Generates a prompt asking for a color based on a given attribute.
prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)

# Asks for a fruit of a specified color.
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)

# Requests the name of a country with a flag containing a certain color.
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)

# Forms a prompt asking for the color of a specific fruit and the flag of a specific country
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

# Extract model message
model_parser = model | StrOutputParser()

# Generating Color
color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)

# Takes a color and uses prompt2 to ask for a corresponding fruit
color_to_fruit = prompt2 | model_parser

# uses prompt3 to find a country with a flag containing that color
color_to_country = prompt3 | model_parser


question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

prompt = question_generator.invoke("warm")
print(prompt)
"""
messages=[HumanMessage(content='What is the color of Coral. and the flag of Comoros?')]
"""

print(model.invoke(prompt))
"""
content='The color of a pomegranate is typically a deep red or maroon. The flag of Armenia consists of three horizontal bands of equal width - the top band is red, the middle band is blue, and the bottom band is orange.'
"""


# Example3
# Branching and Merging
"""
     Input
      / \
     /   \
 Branch1 Branch2
     \   /
      \ /
      Combine
"""
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

print(chain.invoke({"input": "scrum"}))
"""
While Scrum has its pros and cons, it is important to recognize that no project management framework is a one-size-fits-all solution. The cons mentioned should be considered in the context of the specific project and organization.

For example, while Scrum may have a lack of predictability, this can be mitigated by implementing effective estimation techniques and regularly reassessing and adjusting plans. Additionally, while Scrum relies on team communication, organizations can invest in improving communication practices and tools to address any gaps or issues.

Similarly, while Scrum may have limitations for large projects, organizations can adapt Scrum by implementing scaled agile frameworks like SAFe or LeSS to address complexities and dependencies.

Furthermore, while Scrum may prioritize working software over comprehensive documentation, it does not mean that documentation is disregarded entirely. Organizations can establish guidelines and processes to ensure that essential documentation is maintained alongside the iterative development.

Regarding role clarity, organizations can establish clear role definitions and ensure that team members understand their responsibilities and accountabilities. This can be achieved through effective communication and regular feedback.

Lastly, while Scrum relies on experienced Scrum Masters, organizations can invest in training and development programs to enhance the skills and knowledge of Scrum Masters, ensuring effective facilitation of the Scrum process.

In conclusion, while Scrum has its limitations, many of these can be addressed and mitigated through proper implementation, adaptation, and organizational support. It is important to carefully consider the specific project and organizational context to determine if Scrum is the right fit and to make necessary adjustments to maximize its benefits and overcome any potential drawbacks.
"""

In [26]:
import os
from operator import itemgetter
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. 공통 모델 설정 (Llama-3.2 또는 Qwen 사용 권장)
llm_ep = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    max_new_tokens=1024,
    temperature=0.7
)
model = ChatHuggingFace(llm=llm_ep)
model_parser = model | StrOutputParser()

# --- Example 1: Sequential Chaining (순차 체인) ---
#
prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template("what country is the city {city} in? respond in {language}")

chain1 = prompt1 | model_parser
chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model_parser
)

print("--- Example 1 Result ---")
print(chain2.invoke({"person": "obama", "language": "english"}))


# --- Example 2: Parallel Input (병렬 입력 생성) ---
#
prompt1 = ChatPromptTemplate.from_template("generate a {attribute} color. Return the name of the color and nothing else:")
prompt2 = ChatPromptTemplate.from_template("what is a fruit of color: {color}. Return the name of the fruit and nothing else:")
prompt3 = ChatPromptTemplate.from_template("what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:")
prompt4 = ChatPromptTemplate.from_template("What is the color of {fruit} and the flag of {country}?")

color_generator = ({"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser})
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser

question_generator = (
    color_generator
    | {"fruit": color_to_fruit, "country": color_to_country}
    | prompt4
)

print("\n--- Example 2 Result ---")
generated_prompt = question_generator.invoke("warm")
print(f"Generated Question: {generated_prompt.messages[0].content}")
print(model_parser.invoke(generated_prompt))


# --- Example 3: Branching and Merging (분기 및 병합 - Critique Loop) ---
#
planner = (
    ChatPromptTemplate.from_template("Generate a concise argument about: {input}")
    | model_parser
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template("List the pros or positive aspects of {base_response}")
    | model_parser
)
arguments_against = (
    ChatPromptTemplate.from_template("List the cons or negative aspects of {base_response}")
    | model_parser
)

final_responder = (
    ChatPromptTemplate.from_messages([
        ("ai", "{original_response}"),
        ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
        ("system", "Generate a final balanced response given the critique above."),
    ])
    | model_parser
)

full_critique_chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

print("\n--- Example 3 Result (Scrum Analysis) ---")
print(full_critique_chain.invoke({"input": "scrum"}))

--- Example 1 Result ---
The city where Barack Obama, the 44th President of the United States, was born is located in Hawaii, specifically in the state of Hawaii, in the United States.

--- Example 2 Result ---
Generated Question: What is the color of Papaya and the flag of Sri Lanka?
I cannot verify the color of Papaya but the flag of Sri Lanka is predominantly orange, red and green.

--- Example 3 Result (Scrum Analysis) ---
**Scrum: A Balanced Agile Framework**

Scrum is a widely adopted Agile framework that has proven its effectiveness in software development, project management, and other complex endeavors. While it offers numerous benefits, it's essential to acknowledge its potential drawbacks and consider the nuances of its implementation.

**Key Benefits:**

1. **Increased Efficiency**: Scrum's iterative approach allows teams to deliver working software in short cycles, reducing overall project duration and improving response times.
2. **Improved Collaboration**: Scrum fosters 

# Querying a SQL DB

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain.prompts import ChatPromptTemplate
from langchain_community.utilities.sql_database import SQLDatabase


template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

# sqlite3 Chinook.db
# .read Chinook_Sqlite.sql
# download the sql file from the link below
# https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql

db = SQLDatabase.from_uri("sqlite:///./Chinook.db")

def get_schema(_):
    return db.get_table_info()

def run_query(query):
    return db.run(query)

from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

model = ChatOpenAI()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

print(sql_response.invoke({"question": "How many employees are there?"}))
"""
SELECT COUNT(*) FROM Employee
"""

template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: run_query(x["query"]),
    )
    | prompt_response
    | model
)

print(full_chain.invoke({"question": "How many employees are there?"}))
"""
content='There are 8 employees.'
"""

In [28]:
import os
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. 데이터베이스 연결 (sqlite:///./Chinook.db 파일이 있어야 함)
db = SQLDatabase.from_uri("sqlite:///./Chinook.db")

def get_schema(_):
    return db.get_table_info()

def run_query(query):
    # 모델이 생성한 쿼리에 간혹 붙는 백틱(```sql) 등을 제거하는 전처리
    clean_query = query.replace("```sql", "").replace("```", "").strip()
    return db.run(clean_query)

# 2. 모델 설정 (Qwen2.5 또는 Llama-3.2 추천 - 코딩 능력이 좋음)
llm_ep = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    temperature=0.1 # 쿼리 생성은 결정적이어야 하므로 낮은 온도 설정
)
model = ChatHuggingFace(llm=llm_ep)

# 3. SQL 생성 체인 (첫 번째 단계)
# [Image of Text-to-SQL architecture showing natural language query being converted to SQL via LLM and executed on a database]
sql_template = """Based on the table schema below, write ONLY the SQL query that answers the user's question.
Do not write any explanation.

{schema}

Question: {question}
SQL Query:"""
sql_prompt = ChatPromptTemplate.from_template(sql_template)

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | sql_prompt
    | model.bind(stop=["\nSQLResult:", " ;", ";"])
    | StrOutputParser()
)

# 4. 자연어 응답 체인 (두 번째 단계)
# [Image of SQL-to-Text process showing database results being formatted into a natural language response by an AI model]
response_template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}
Natural Language Response:"""
response_prompt = ChatPromptTemplate.from_template(response_template)

full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: run_query(x["query"]),
    )
    | response_prompt
    | model
    | StrOutputParser()
)

# 5. 실행 및 확인
try:
    print("--- 1. 생성된 SQL 쿼리 ---")
    query_text = sql_response.invoke({"question": "How many employees are there?"})
    print(query_text)

    print("\n--- 2. 최종 자연어 답변 ---")
    final_answer = full_chain.invoke({"question": "How many employees are there?"})
    print(final_answer)
except Exception as e:
    print(f"Error: {e}")

--- 1. 생성된 SQL 쿼리 ---
SELECT COUNT(*) FROM Employee

--- 2. 최종 자연어 답변 ---
There are 8 employees in the database.


# Code Generator

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
)
from langchain.schema import StrOutputParser
from langchain_community.utilities import PythonREPL

# Introduce PythonREPL
python_repl = PythonREPL()
print(python_repl.run("print(1+1)"))
"""
Python REPL can execute arbitrary code. Use with caution.
2
"""

template = """Write some python code to solve the user's problem.

Return only python code in Markdown format, e.g.:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

model = ChatOpenAI()

def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

chain = prompt | model | StrOutputParser()

print(chain.invoke({"input": "Write the function to sort the list. Then call the function by pasing [1,4,2]"}))
"""
```python
def sort_list(lst):
    return sorted(lst)

my_list = [1, 4, 2]
sorted_list = sort_list(my_list)
print(sorted_list)
```
"""
repl_chain = chain | _sanitize_output | PythonREPL().run

print(repl_chain.invoke({"input": "Write the function to sort the list. Then call the function by pasing [1,4,2]"}))
"""
Python REPL can execute arbitrary code. Use with caution.
[1, 2, 4]
"""

In [5]:
pip install langchain-experimental

  Using cached langchain_core-1.2.8-py3-none-any.whl.metadata (3.7 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_text_splitters-1.1.0-py3-none-any.whl.metadata (2.7 kB)
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)
Using cached langchain_core-1.2.8-py3-none-any.whl (495 kB)
Using cached langchain_text_splitters-1.1.0-py3-none-any.whl (34 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.83
    Uninstalling langchain-core-0.3.83:
      Successfully uninstalled langchain-core-0.3.83
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.31
    Uninstalling langchain-community-0.3.31:
      Successfu

In [5]:
import os
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# [중요] 임포트 경로를 experimental로 변경합니다.
from langchain_experimental.utilities import PythonREPL

# 1. 모델 설정 (Qwen2.5-7B 추천)
llm_ep = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    max_new_tokens=1024,
    temperature=0.1
)
model = ChatHuggingFace(llm=llm_ep)

# 2. 프롬프트 설정
template = """Write some python code to solve the user's problem.

Return only python code in Markdown format, e.g.:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

# 3. 출력 정제 함수
def _sanitize_output(text: str):
    if "```python" in text:
        _, after = text.split("```python")
        return after.split("```")[0].strip()
    return text.strip()

# 4. 체인 구성
code_gen_chain = prompt | model | StrOutputParser()

# PythonREPL 객체 생성 (실행 시 주의 문구가 출력됩니다)
repl = PythonREPL()
repl_chain = code_gen_chain | _sanitize_output | repl.run

# 5. 실행 테스트
input_query = "Write a function to sort a list. Then call the function by passing [1, 4, 2] and print the result."

print("--- 1. 생성된 코드 확인 ---")
generated_code = code_gen_chain.invoke({"input": input_query})
print(generated_code)

print("\n--- 2. 실행 결과 확인 ---")
#
execution_result = repl_chain.invoke({"input": input_query})
print(execution_result)

--- 1. 생성된 코드 확인 ---
```python
def sort_list(lst):
    return sorted(lst)

result = sort_list([1, 4, 2])
print(result)
```

--- 2. 실행 결과 확인 ---


[1, 2, 4]

